In [22]:
import findspark
findspark.init()

from pyspark.sql import SparkSession


spark = SparkSession.builder \
    .appName("Read JSON from S3") \
    .config("spark.hadoop.fs.s3a.access.key", "YOUR_ACCESS_KEY") \
    .config("spark.hadoop.fs.s3a.secret.key", "YOUR_SECRET_KEY") \
    .config("spark.hadoop.fs.s3a.endpoint", "http://localhost:4566") \
    .getOrCreate()

# Replace with your S3 bucket and file path
s3_path = "C://users/bindu/Downloads/Q4nbwea22jo.json"


In [23]:

# Read JSON file
df = spark.read.json(s3_path)

# Display the schema
df.printSchema()


root
 |-- author: string (nullable = true)
 |-- likeCount: long (nullable = true)
 |-- publishedAt: string (nullable = true)
 |-- text: string (nullable = true)



In [24]:

# Show the first few rows
df.show()


+--------------------+---------+--------------------+--------------------+
|              author|likeCount|         publishedAt|                text|
+--------------------+---------+--------------------+--------------------+
|    @mychannel-tt9ib|        0|2024-08-02T12:04:57Z|how to register a...|
| @mageshselvaraj7869|        0|2024-08-01T15:57:37Z|My friend suggest...|
| @AshwaniKumar-zg9jx|        0|2024-07-09T07:10:46Z|Have completed th...|
|    @mohanavamsi1659|        0|2024-07-07T10:37:48Z|hello sir i dont ...|
|@YashKarambalkar-...|        0|2024-06-17T18:21:17Z|Best video for st...|
|   @projectveritas93|        0|2024-05-28T04:33:46Z|HI SIR. CAN AUTOM...|
|     @gowthams.v6004|        1|2024-05-26T06:08:24Z|Hi  summit sir wh...|
|      @sutapakar7522|        0|2024-05-08T14:32:33Z|I completed SQL 9...|
|           @Ashimann|        1|2024-05-02T04:52:43Z|i want to learn p...|
|@chamarthschandu9050|        0|2024-04-29T06:56:12Z|Hello sir <br>It ...|
|     @manojbobby1284|   

In [40]:
from pyspark.sql.functions import to_date,count
from pyspark.sql.window import Window
# Perform transformations
transformed_df = df.select("publishedAt", "likecount").filter(df.text != '')
dt_df=transformed_df.withColumn("dt",to_date("publishedAt")).drop("publishedAt")
wdf =Window.partitionBy("dt").orderBy("dt")
final_df = dt_df.withColumn("total_likes", count("likecount").over(wdf)).drop("likecount").dropDuplicates()
final_df.printSchema()

spark.sparkContext.setLogLevel("DEBUG")
final_df.write.partitionBy("dt").mode("overwrite").csv("C://users/bindu/Downloads/output/")



root
 |-- dt: date (nullable = true)
 |-- total_likes: long (nullable = false)



Py4JJavaError: An error occurred while calling o775.csv.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.jobAbortedError(QueryExecutionErrors.scala:651)
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:278)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:186)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:113)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:111)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.executeCollect(commands.scala:125)
	at org.apache.spark.sql.execution.QueryExecution$$anonfun$eagerlyExecuteCommands$1.$anonfun$applyOrElse$1(QueryExecution.scala:98)
	at org.apache.spark.sql.execution.SQLExecution$.$anonfun$withNewExecutionId$6(SQLExecution.scala:109)
	a